In [14]:
import torch
import math

In [10]:
def rotate_towards(a, b, gamma):
    """
    Rotates tensor 'a' towards tensor 'b' by angle gamma (in degrees).
    """
    shape = a.size()
    a = torch.flatten(a)
    b = torch.flatten(b)

    cos_theta = torch.dot(a, b) / (torch.norm(a) * torch.norm(b))
    print("cos_theta", cos_theta)
    theta = torch.acos(torch.clamp(cos_theta, -1.0, 1.0))
    gamma_rad = torch.deg2rad(torch.tensor(gamma))
    
    if theta < gamma_rad:
        return b  # If gamma exceeds the angle between, return b
    
    direction = b - a
    direction = direction / torch.norm(direction)  # Normalize direction
    rotated_a = a + torch.norm(a) * torch.tan(gamma_rad) * direction
    
    return rotated_a.view(shape)

In [21]:
a = torch.tensor([1.0,0.0])
b = torch.tensor([0.0,1.0])
gamma = 90

In [22]:
rotate_towards(a, b, gamma)

cos_theta tensor(0.)


tensor([ 16176717., -16176716.])

In [19]:
gamma_rad = torch.deg2rad(torch.tensor(180))

In [20]:
gamma_rad 

tensor(3.1416)

In [44]:
import torch

import torch

def rotate_towards(a: torch.Tensor, b: torch.Tensor, gamma: float) -> torch.Tensor:
    """
    Rotates tensor 'a' towards tensor 'b' by an angle 'gamma' (in degrees) in N-dimensional space.

    Parameters:
        a (torch.Tensor): The input tensor to be rotated.
        b (torch.Tensor): The target tensor to rotate towards.
        gamma (float): The rotation angle in degrees.

    Returns:
        torch.Tensor: The rotated tensor.
    """
    if torch.count_nonzero(a)==0:
        return a
    
    shape = a.size()
    a = torch.flatten(a)
    b = torch.flatten(b)
    
    # Normalize vectors
    a = a / a.norm(dim=-1, keepdim=True)
    b = b / b.norm(dim=-1, keepdim=True)
    
    # Compute the rotation axis (perpendicular component of b relative to a)
    v = b - (a * (a * b).sum(dim=-1, keepdim=True))  # Remove parallel component
    v_norm = v.norm(dim=-1, keepdim=True)
    
    # If v_norm is zero, a and b are already aligned, return a
    if torch.all(v_norm < 1e-8):
        return a

    v = v / v_norm  # Normalize rotation axis
    
    # Compute cosine and sine of the rotation angle
    gamma_rad = torch.deg2rad(torch.tensor(gamma, dtype=a.dtype, device=a.device))
    cos_gamma = torch.cos(gamma_rad)
    sin_gamma = torch.sin(gamma_rad)
    
    # Rodrigues' rotation formula
    rotated_a = cos_gamma * a + sin_gamma * v + (1 - cos_gamma) * (v * (v * a).sum(dim=-1, keepdim=True))

    return (rotated_a * a.norm(dim=-1, keepdim=True)).view(shape)  # Rescale to original magnitude

# Test cases
a_2d = torch.tensor([1.0, 0.0])
b_2d = torch.tensor([0.0, 1.0])
gamma = 90

a_3d = torch.tensor([1.0, 0.0, 0.0])
b_3d = torch.tensor([0.0, 1.0, 0.0])

print(rotate_towards(a_2d, b_2d, gamma))  # Expected ~[0, 1]
print(rotate_towards(a_3d, b_3d, gamma))  # Expected ~[0, 1, 0]




tensor([-4.3711e-08,  1.0000e+00])
tensor([-4.3711e-08,  1.0000e+00,  0.0000e+00])


In [47]:
# Test case
a = torch.tensor([[0.0, 0.0], [0.0, 0.0]])
b = torch.tensor([[0.0, 1.0], [0.0, 0.0]])
gamma = 3600

rotated_a = rotate_towards(a, b, gamma)
print(rotated_a)  # Expected output: tensor([0.0, 1.0])

tensor([[nan, nan],
        [nan, nan]])


In [49]:
a = torch.tensor([
    [[ 2.9963e-04,  3.9493e-04, -5.2467e-05],
              [ 1.3937e-04,  1.8504e-04, -2.4397e-04],
          [ 2.0497e-04,  1.0249e-03,  8.5719e-04]],

         [[ 2.8294e-06,  1.2447e-04,  1.5370e-04],
          [ 9.8873e-05,  8.0010e-05,  3.6165e-04],
          [ 4.3490e-04, -7.6380e-05,  1.0160e-03]],

         [[ 3.9777e-04,  5.5658e-04,  4.1153e-04],
          [ 4.3705e-04,  6.8596e-04,  9.1332e-04],
          [ 2.2702e-04,  1.0338e-03,  1.7728e-03]]]
)

In [56]:
rotated_a = rotate_towards(a, torch.rand(a.shape), 1)

In [57]:
rotated_a

tensor([[[ 0.0976,  0.1247, -0.0156],
         [ 0.0475,  0.0612, -0.0737],
         [ 0.0711,  0.3260,  0.2720]],

        [[ 0.0087,  0.0403,  0.0493],
         [ 0.0352,  0.0311,  0.1174],
         [ 0.1362, -0.0222,  0.3168]],

        [[ 0.1247,  0.1772,  0.1325],
         [ 0.1385,  0.2162,  0.2913],
         [ 0.0782,  0.3276,  0.5564]]])

In [7]:
import torch
import torch.nn.functional as F

def closest_cosine_similarity(v, tensor_set):
    """
    Compute the highest cosine similarity between v and any tensor in tensor_set.
    
    Args:
        v (torch.Tensor): The target tensor.
        tensor_set (torch.Tensor): A 2D tensor where each row is a candidate tensor.
    
    Returns:
        float: The maximum cosine similarity value.
    """
    # Normalize vectors
    v_norm = F.normalize(v, dim=0)  # Shape: (d,)
    tensor_set_norm = F.normalize(tensor_set, dim=1)  # Shape: (m, d)

    # Compute cosine similarities
    cos_sim = torch.matmul(tensor_set_norm, v_norm)  # Shape: (m,)
    
    # Get the maximum similarity
    return torch.max(cos_sim).item()

# Example usage
v = torch.tensor([1.0, 2.0, 3.0])
tensor_set = torch.tensor([[1.1, 2.1, 3.1], 
                           [1.7, 2.2, 3.0], 
                           [0.5, 2.0, 1.5]])  # Each row is a candidate tensor

max_cos_sim = closest_cosine_similarity(v, tensor_set)
print("Closest cosine similarity:", max_cos_sim)


Closest cosine similarity: 0.9998592734336853


In [1]:
import torch
import torch.nn.functional as F

def closest_tensor_cosine_similarity(v, tensor_set):
    """
    Find the tensor in tensor_set that has the highest cosine similarity with v
    and return both the highest similarity and the closest tensor.
    
    Args:
        v (torch.Tensor): The target tensor.
        tensor_set (torch.Tensor): A 2D tensor where each row is a candidate tensor.
    
    Returns:
        float: The maximum cosine similarity value.
        torch.Tensor: The closest tensor.
    """
    # Normalize vectors
    v_norm = F.normalize(v, dim=0)  # Shape: (d,)
    tensor_set_norm = F.normalize(tensor_set, dim=1)  # Shape: (m, d)

    # Compute cosine similarities
    cos_sim = torch.matmul(tensor_set_norm, v_norm)  # Shape: (m,)

    # Get the index of the maximum similarity
    max_index = torch.argmax(cos_sim).item()
    
    # Retrieve the closest tensor
    closest_tensor = tensor_set[max_index]
    
    return cos_sim[max_index].item(), closest_tensor

# Example usage
v = torch.tensor([1.0, 2.0, 3.0])
tensor_set = torch.tensor([[0.9, 2.1, 3.1], 
                           [1.1, 2.2, 3.0], 
                           [0.5, 1.0, 1.5]])  # Each row is a candidate tensor

max_cos_sim, closest_tensor = closest_tensor_cosine_similarity(v, tensor_set)
print("Closest cosine similarity:", max_cos_sim)
print("Closest tensor:", closest_tensor)


Closest cosine similarity: 0.9999998807907104
Closest tensor: tensor([0.5000, 1.0000, 1.5000])
